## Installing Important Packages

In [ ]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install jiwer
!pip install kenlm
!pip install pyctcdecode

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


# Section 1: Automatic Speech Recognition

## 1.1 Import important libraries

In [5]:
from datasets import Dataset, Audio
from transformers import pipeline
import pandas as pd
import evaluate
import torch
import time

## 1.2 Load Urdu Recipes dataset

In [6]:
df = pd.read_excel('new-urdu-recipes.xlsx')
df.head()

,Id,recipe_name,additional_phrase,ing_name,Ing_quantity,Ing_unit,source,audio
0,1,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء,NaN,NaN,NaN,kfoods,dataset\Balochi Gosht - kfoods.wav
1,1,بلوچی گوشت,NaN,بون لیس مٹن,½,کلو,kfoods,dataset\Balochi Gosht - kfoods.wav
2,1,بلوچی گوشت,NaN,ادرک لہسن کا پیسٹ,1,کھانے کا چمچہ,kfoods,dataset\Balochi Gosht - kfoods.wav
3,1,بلوچی گوشت,NaN,ثابت لال مرچ,6,عدد,kfoods,dataset\Balochi Gosht - kfoods.wav
4,1,بلوچی گوشت,NaN,ہری مرچ,3,عدد,kfoods,dataset\Balochi Gosht - kfoods.wav


## 1.3 Preprocessing dataset

In [7]:
df['Ing_quantity'] = df['Ing_quantity'].replace(r' ', '', regex= True)
df['ing_name'] = df['ing_name'].str.strip('\t')
df = df.fillna('').astype(str)
df = df.replace(r'[\n\xa0\u200c]', '', regex = True)

### Converting numeric quantities into words

In [8]:
urdu_counting = {'0.25': 'ایک چوتھائی', '0.5': 'آدھا', '½-1': 'ڈیڑھ','1½': 'ڈیڑھ', '½-2': 'ڈھائی',
 '2½': 'ڈھائی', '½': 'آدھا', '1': 'ایک', '2': 'دو', '3': 'تین', '4': 'چار', '5': 'پانچ', '6': 'چھ',
 '7': 'سات', '8': 'آٹھ', '9': 'نو', '10': 'دس', '11': 'گیارہ', '12': 'بارہ', '13': 'تیرہ', '14': 'چودہ',
 '15': 'پندرہ', '16': 'سولہ', '17': 'سترہ', '18': 'اٹھارہ', '19': 'انیس', '20': 'بیس', '21': 'اکیس', '22': 'بائیس',
 '23': 'تئیس', '24': 'چوبیس', '25': 'پچیس', '26': 'چھببیس', '27': 'ستائیس', '28': 'اٹھائیس', '29': 'انتیس', '30': 'تیس',
 '31': 'اکتیس', '32': 'بتیس', '33': 'تینتیس', '34': 'چونتیس', '35': 'پینتیس', '36': 'چھتیس', '37': 'سینتیس', '38': 'اڑتیس',
 '39': 'انتالیس', '40': 'چالیس', '41': 'اکتالیس', '42': 'بیالیس', '43': 'تینتالیس', '44': 'چوالیس', '45': 'پینتالیس', '46': 'چھیالیس',
 '47': 'سینتالیس', '48': 'اڑتالیس', '49': 'انچاس', '50': 'پچاس', '51': 'اکیاون', '52': 'باون', '53': 'ترپن', '54': 'چون', 
 '55': 'پچپن', '56': 'چھپن', '57': 'ستاون', '58': 'اٹھاون', '59': 'انسٹھ', '60': 'ساٹھ', '61': 'اکسٹھ', '62': 'باسٹھ',
 '63': 'ترسٹھ', '64': 'چوسٹھ', '65': 'پینسٹھ', '66': 'چھیاسٹھ', '67': 'سڑسٹھ', '68': 'اٹھسٹھ', '69': 'انہتر', '70': 'ستر',
 '71': 'اکہتر', '72': 'بہتر', '73': 'تہتر', '74': 'چوہتر', '75': 'پچہتر', '76': 'چھہتر', '77': 'ستتر', '78': 'اٹھہتر',
 '79': 'اناسی', '80': 'اسی', '81': 'اکیاسی', '82': 'بیاسی', '83': 'تراسی', '84': 'چوراسی', '85': 'پچاسی', '86': 'چھیاسی',
 '87': 'ستاسی', '88': 'اٹھاسی', '89': 'نواسی', '90': 'نوے', '91': 'اکانوے', '92': 'بانوے', '93': 'ترانوے', '94': 'چورانوے',
 '95': 'پچانوے', '96': 'چھیانوے', '97': 'ستانوے', '98': 'اٹھانوے', '99': 'ننانوے', '100': 'سو', '150': 'ایک سو پچاس', '200': 'دو سو',
 '250': 'دو سو پچاس', '300': 'تین سو', '350': 'تین سو پچاس', '400': 'چار سو',
 '450': 'چار سو پچاس', '500': 'پانچ سو'}

In [9]:
def num_to_words(value): 
    if '-' in value:
        tmp = value.split('-')
        if tmp[0] != '½':
            tmp[0] = urdu_counting[tmp[0]]
            tmp[1] = urdu_counting[tmp[1]]
            return (' سے '.join(tmp))           
    
    if value in urdu_counting.keys():
        return urdu_counting[value]
    return  value
for i in df.index:
    df.loc[i, 'Ing_quantity'] = num_to_words(df.loc[i, 'Ing_quantity'])
    df.loc[i, 'ing_name'] = df.loc[i, 'ing_name'].replace('14', num_to_words('14'))
    df.loc[i, 'ing_name'] = df.loc[i, 'ing_name'].replace('16', num_to_words('16'))

### Combine necessary columns to create one complete Ingredient Phrase for each recipe

In [10]:
df['text'] = df[['additional_phrase', 'ing_name', 'Ing_quantity', 'Ing_unit']].astype(str).apply(' '.join, axis=1)
chars_to_remove_regex = r'[\ ِ\'ُ\،\?\.\!\-\;\:\"\“\%\‘\”\�\'\)\(]| +\n'
df['text'] = df['text'].replace(chars_to_remove_regex, ' ', regex = True).replace(r' +', ' ', regex = True)
df['audio'] = df["audio"].replace(r"\\", "/", regex = True)
df.head()

,Id,recipe_name,additional_phrase,ing_name,Ing_quantity,Ing_unit,source,audio,text
0,1,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء,,,,kfoods,dataset/Balochi Gosht - kfoods.wav,بلوچی گوشت بنانے کے اجزاء
1,1,بلوچی گوشت,,بون لیس مٹن,آدھا,کلو,kfoods,dataset/Balochi Gosht - kfoods.wav,بون لیس مٹن آدھا کلو
2,1,بلوچی گوشت,,ادرک لہسن کا پیسٹ,ایک,کھانے کا چمچہ,kfoods,dataset/Balochi Gosht - kfoods.wav,ادرک لہسن کا پیسٹ ایک کھانے کا چمچہ
3,1,بلوچی گوشت,,ثابت لال مرچ,چھ,عدد,kfoods,dataset/Balochi Gosht - kfoods.wav,ثابت لال مرچ چھ عدد
4,1,بلوچی گوشت,,ہری مرچ,تین,عدد,kfoods,dataset/Balochi Gosht - kfoods.wav,ہری مرچ تین عدد


In [11]:
new_df = df.groupby(['Id', 'recipe_name', 'audio'])['text'].apply('، '.join).reset_index()

In [12]:
new_df.head()

,Id,recipe_name,audio,text
0,1,بلوچی گوشت,dataset/Balochi Gosht - kfoods.wav,بلوچی گوشت بنانے کے اجزاء ، بون لیس مٹن آدھا ...
1,10,فروٹی اسموتھی,dataset/Fruity Smoothie - kfoods.wav,فروٹی اسموتھی بنانے کے اجزاء ، آڑو ایک کپ، ...
2,11,کرسپ اسپنچ وِد سیلیڈ,dataset/Crisp Chicken with Salad - kfoods.wav,کرسپ اسپنچ و د سیلیڈ بنانے کے اجزاء ، چکن بری...
3,12,کریم آلو,dataset/Cream Aloo - kfoods.wav,کریم آلو بنانے کے اجزاء ، آلو ایک کلو، ہری پ...
4,13,انڈے کی بریانی,dataset/Anday ki biryani.wav,انڈے کی بریانی بنانے کے اجزاء ، ابلے انڈے آٹھ...


# Section 2: NER on Urdu Recipes

In [16]:
import spacy
import re
import pandas as pd

In [17]:
nlp = spacy.load("urdu_ner_model")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:887: UserWarning: [W095] Model 'ur_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [18]:
predictions = pd.read_csv('Predictions.csv')

In [19]:
predictions['Whisper Predicted Text'] = predictions["Whisper Predicted Text"].replace(r'،', '', regex = True)
predictions['Original Text'] = new_df["text"][:50]
urdu_numbers = {v: k for k, v in urdu_counting.items()}
predictions = predictions.replace(urdu_numbers, regex = True)

In [20]:
predictions.head()

,Unnamed: 0,Recipe Name,Original Text,Whisper Predicted Text,Wav2Vec2 Predicted Text
0,0,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء ، بون لیس مٹن ½ کلو...,بلوچی گوشت بنانے کے چمچے تیل 3 کھانے کے چمچے پ...,بلوچی گوشت بنانے کے سا مول لیس مٹ دھکیل ادرک ل...
1,1,فروٹی اسموتھی,فروٹی اسموتھی بنانے کے اجزاء ، آڑو 1 کپ، در...,فروٹی سمودی بنانے کے اتزا اڑو 1 کپ ترمیانے گیل...,فروٹی اس مودی بنانے کے عجز اڑو 1 کپ درمیانے کی...
2,2,کرسپ اسپنچ وِد سیلیڈ,کرسپ اسپنچ و د سیلیڈ بنانے کے اجزاء ، چکن بری...,کرسپ سپنچ ویٹ سالٹ بنانے کے اجزاء چکن بریسٹ 2 ...,کرسپسپنچوت سالٹ بنانے کے از چکن برس 2 عدد کھیر...
3,3,کریم آلو,کریم آلو بنانے کے اجزاء ، آلو 1 کلو، ہری پیا...,کریم آلو بنانے کے عدزہ آلو ½ کلو ہری پیاز 2 عد...,کریم آلو بنانے کے عصا آلو ادا کیلوں ہری پیاز 2...
4,4,انڈے کی بریانی,انڈے کی بریانی بنانے کے اجزاء ، ابلے انڈے 8 ع...,انڈے کی پیانی گھیتزا ابلے انڈے آٹ اتد چاول 2 ...,انڈے کی بیانی کہ عدزاعبلےانڈے اٹھتر چاول 2 پال...


In [21]:
predictions['Whisper Predicted Text'][0]

'بلوچی گوشت بنانے کے چمچے تیل 3 کھانے کے چمچے پیاز 1 ادد جائفل جوتری پاوڈر آدہ چائے کا چمچہ شملا مرچ 1 ادد کڑی پتے چند ادد پودینا 1 چوتائی کٹی پسی لال مرچ 1 چائے کا چمچہ ہلدی 1 چائے کا چمچہ ابلی کٹے ٹیمارٹر 2 ادد'

In [22]:
tagged_entities = []
def tag_input(input_str):
  org_string = input_str 
  # Define the regex pattern
  pattern = r'(\S+)\s*([\d\u00BC-\u00BE\u2150-\u215E]+(?:\.[\d\u00BC-\u00BE\u2150-\u215E]+)?)\s*(\S+)'
  # Extract the matches using the pattern
  matches = re.findall(pattern, input_str)
  decreased_length = 0
  annotations = {'entities': []}
  # Create a list of tuples with the tagged entities
  for match in matches:
    ingredient = re.search(match[0].strip(), input_str)
    quantity = re.search(match[1].strip(), input_str)
    unit = re.search(match[2].strip(), input_str)
    annotations['entities'] += [(ingredient.start() + decreased_length, ingredient.end() + decreased_length, "Ingredient"), 
                                          (quantity.start() + decreased_length, quantity.end() + decreased_length, "Quantity"), 
                                          (unit.start() + decreased_length, unit.end() + decreased_length, "Unit")]
    input_str = input_str[unit.end()+1:]
    decreased_length = unit.end()
  tagged_entities.append((org_string, annotations))

In [23]:
for i in predictions['Whisper Predicted Text']:
    tag_input(i) 

In [ ]:
tagged_entities

[('بلوچی گوشت بنانے کے چمچے تیل 3 کھانے کے چمچے پیاز 1 ادد جائفل جوتری پاوڈر آدہ چائے کا چمچہ شملا مرچ 1 ادد کڑی پتے چند ادد پودینا 1 چوتائی کٹی پسی لال مرچ 1 چائے کا چمچہ ہلدی 1 چائے کا چمچہ ابلی کٹے ٹیمارٹر 2 ادد',
  {'entities': [(25, 28, 'Ingredient'),
    (29, 30, 'Quantity'),
    (31, 36, 'Unit'),
    (44, 48, 'Ingredient'),
    (49, 50, 'Quantity'),
    (51, 54, 'Unit'),
    (59, 62, 'Ingredient'),
    (63, 64, 'Quantity'),
    (65, 68, 'Unit'),
    (66, 72, 'Ingredient'),
    (73, 74, 'Quantity'),
    (75, 81, 'Unit'),
    (43, 46, 'Ingredient'),
    (47, 48, 'Quantity'),
    (49, 53, 'Unit'),
    (30, 34, 'Ingredient'),
    (35, 36, 'Quantity'),
    (37, 41, 'Unit'),
    (36, 43, 'Ingredient'),
    (44, 45, 'Quantity'),
    (46, 49, 'Unit')]}),
 ('فروٹی سمودی بنانے کے اتزا اڑو 1 کپ ترمیانے گیلے 2 اتد دہی 1 کپ برف 1 کپ چینی 1 کھانے کا چمچا',
  {'entities': [(26, 29, 'Ingredient'),
    (30, 31, 'Quantity'),
    (32, 34, 'Unit'),
    (42, 46, 'Ingredient'),
    (47, 48, 'Quantit

In [25]:
# Function to evaluate the model on a test set
def evaluate_model(test_data):
    # Initialize counters
    total_true = 0
    total_predicted = 0
    total_correct = 0
    
    # Iterate over the test data
    for text, annotations in test_data:
        # Get the true entities
        true_entities = set([(ent[0], ent[1], ent[2]) for ent in annotations["entities"]])
        
        
        # Get the predicted entities
        doc = nlp(text)
        predicted_entities = set([(ent.text, ent.label_) for ent in doc.ents])
        print(predicted_entities)
        
        # Update the counters
        total_true += len(true_entities)
        total_predicted += len(predicted_entities)
        total_correct += len(true_entities.intersection(predicted_entities))
    
    # Calculate precision, recall, and F1 score
    precision = total_correct / total_predicted if total_predicted > 0 else 0.0
    recall = total_correct / total_true if total_true > 0 else 0.0
    f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0.0
    
    # Print the evaluation metrics
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 score: {f1_score:.2f}")

# Evaluate the model on the test data
evaluate_model(tagged_entities)

{('جائفل جوتری پاوڈر', 'Ingredient'), ('ادد پودینا', 'Ingredient'), ('ٹیمارٹر', 'Ingredient'), ('چائے کا چمچہ شملا مرچ', 'Ingredient'), ('1', 'Quantity'), ('بلوچی گوشت بنانے کے چمچے تیل', 'Ingredient'), ('چائے کا چمچہ', 'Unit'), ('2', 'Quantity'), ('پیاز', 'Ingredient'), ('3', 'Quantity'), ('ادد', 'Unit'), ('کھانے کے چمچے', 'Unit'), ('چوتائی کٹی پسی', 'Unit')}
{('کھانے کا چمچا', 'Unit'), ('1', 'Quantity'), ('فروٹی سمودی بنانے کے اتزا اڑو', 'Ingredient'), ('2', 'Quantity'), ('ترمیانے گیلے', 'Ingredient'), ('کپ', 'Unit'), ('اتد', 'Unit'), ('چینی', 'Ingredient'), ('برف', 'Ingredient')}
{('خیرہ باری سلائز', 'Ingredient'), ('ٹیمارٹر', 'Ingredient'), ('ائیز برک', 'Ingredient'), ('1', 'Quantity'), ('چائے کا چمچہ', 'Unit'), ('پول باری چینی', 'Ingredient'), ('2', 'Quantity'), ('کٹی', 'Unit'), ('آدہ', 'Quantity'), ('ادد', 'Unit'), ('باری کٹی پالک', 'Ingredient'), ('کرسپ سپنچ ویٹ سالٹ بنانے کے اجزاء چکن بریسٹ', 'Ingredient')}
{('½', 'Quantity'), ('4', 'Quantity'), ('2', 'Quantity'), ('کپ', 'Unit